In [7]:
from urllib.parse import urlparse
from sqlalchemy import create_engine
import pandas as pd


#dbserver_sampleの中身を書き換えて、ファイル名をdbserver.txtに変更
#dbserver.txtからdbのpathを読み込む

db_path = open('dbserver_sample.txt', 'r').read()

In [ ]:
#dbに接続し、テーブルから取ってきたデータをdataframeに変換
from sqlalchemy.sql import text
from datetime import datetime, date, timezone, timedelta

url = urlparse(db_path) 

conn = create_engine('mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8'.format(host = url.hostname, port=url.port, user = url.username, password= url.password, database = url.path[1:]),encoding='utf-8')

table='raw_from_api'
# sql_read=text("select * from qiita_db.raw_from_api where likes_count = {};".format(640))
#sql_read=text("select * from qiita_db.raw_from_api where created_at = '{}';".format('2019-01-01 10:53:44'))
sql_read=text("select * from qiita_db.raw_from_api where created_at between'{}' and '{}';".format('2019-01-01 00:00:00','2019-02-01 00:00:00'))

df = pd.read_sql(sql_read, conn)

In [23]:
df.head(3)

,id,article_id,title,created_at,updated_at,likes_count,comments_count,tags_str,user_id,user_permanent_id,url,html,tokens,similar_articles
0,469421,a0ae89bb2ff08c50d4ec,Mac上でWindowsのコマンドラインアプリをビルドする,2019-01-01 09:29:37,2019-01-01 09:29:37,0,0,"C,Mac,MacOSX,wine,MinGW",ossan-buhibuhi,101524,https://qiita.com/ossan-buhibuhi/items/a0ae89b...,<p>Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、...,Chicken Scheme が ファイル パス 関係 で どう 動作 する か 確認 し ...,None
1,469420,298d3bf877b73ce50612,Webアプリ開発を試したいJSライブラリ,2019-01-01 09:33:15,2019-01-01 09:33:15,3,0,JavaScript,giogiokuma,60856,https://qiita.com/giogiokuma/items/298d3bf877b...,<p>自分が試したいJSライブラリをメモ（2019）<br>\n<strong><a hre...,自分 が 試し たい JS ライブラリ を メモ （ 2019 ） オーディオスプライト に...,None
2,469419,2bad5f7dda7c35afc9fa,Qiitaで自分のContributinランキングが分かる最強システムの秘密とは‥？ RT ...,2019-01-01 09:55:56,2019-01-01 09:56:15,0,0,"Qiita,Twitter",YumaInaura,89618,https://qiita.com/YumaInaura/items/2bad5f7dda7...,"<p><a href=""https://mogya.github.io/qiitaRank/...","エンジニア ランキング vQiita の contributions 値 から , 日本 で...",None


In [4]:
len(df)

8248

In [8]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
import MeCab
import random
from gensim import corpora, models
import pandas as pd
from tqdm import tqdm

In [9]:
## 分かち書き
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]

class Tokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
            self.parser = mecab.parse
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords]
        return res

In [10]:
import pandas as pd
from typing import Tuple
import lxml.html
import re

# htmlから文章を抽出
def get_string(html_text):
    html = lxml.html.fromstring(html_text)
    remove_tags = ('.//style', './/script', './/noscript')
    for remove_tag in remove_tags:
        for tag in html.findall(remove_tag):
            tag.drop_tree()
            # ここでの削除は元の変数tに反映されます。
    
    codeframe_list = []
    lang_list = []
    # コードの削除
    for tag in html.findall(".//div[@class='code-frame']"):
        codeframe_list.append(tag.text_content())
        lang_list.append(tag.attrib["data-lang"])
        tag.drop_tree()

    atext_list = []
    ahref_list = []
    # href　リンクの削除
    for tag in html.cssselect('a'):
        if tag.text is not None:
            atext_list.append(tag.text)
        if tag.get('href') is not None:
            ahref_list.append(tag.get('href'))
        tag.drop_tree()
        
    code_list = []
    # 一行コードの削除    
    for cc in  html.cssselect('code'):
        if cc.text is not None:
            code_list.append(cc.text)
        cc.drop_tree()
    
    text = html.text_content().strip('\n')

    return  pd.Series(["".join(text.split('\n')), ",".join(codeframe_list), ",".join(lang_list), ",".join(code_list), ",".join(atext_list), ",".join(ahref_list)], index=['text', 'code-frame', 'lang', 'code', 'a-text', 'a-href'])



In [11]:
# 文章から分かち書きする
def get_shareing(html_text):
    mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
    t = Tokenizer(stopwords + ["…。"] , mecab.parse, exclude_reg=r"\d(年|月|日)")

    seri = get_string(html_text)
    return t.tokenize(seri['text'])

In [12]:
share = []
for index, row in df.iterrows():
    share.append(", ".join(get_shareing(row['html'])))
df_text = df
df_text["sharing"] = share

NameError: name 'df' is not defined

In [7]:
# df_text = pd.DataFrame(columns=['id', 'text', 'code-frame', 'lang', 'code', 'a-text', 'a-href'])
# for index, row in df.iterrows():
#     seri = get_string(row['html'])
#     ss = pd.concat([seri, row])
# #     seri['id'] = row['id']
#     df_text = df_text.append(ss, ignore_index=True)
# df_text = df_text.drop(["html", "updated_at", "tokens", "user_id", "user_permanent_id", "comments_count", "code-frame", "code", "a-text", "a-href", "lang"], axis=1)

In [19]:
df_text_ed = df_text.drop(["html", "updated_at", "tokens", "user_id", "user_permanent_id", "comments_count", "similar_articles"], axis=1)
df_text_ed.head(3)

,id,article_id,title,created_at,likes_count,tags_str,url,sharing
0,469421,a0ae89bb2ff08c50d4ec,Mac上でWindowsのコマンドラインアプリをビルドする,2019-01-01 09:29:37,0,"C,Mac,MacOSX,wine,MinGW",https://qiita.com/ossan-buhibuhi/items/a0ae89b...,"chicken, scheme, ファイル, パス, 動作, する, 確認, する, ん, ..."
1,469420,298d3bf877b73ce50612,Webアプリ開発を試したいJSライブラリ,2019-01-01 09:33:15,3,JavaScript,https://qiita.com/giogiokuma/items/298d3bf877b...,"試す, JS, ライブラリ, メモ, オーディオスプライト, 制御, 可能, オーディオ, ..."
2,469419,2bad5f7dda7c35afc9fa,Qiitaで自分のContributinランキングが分かる最強システムの秘密とは‥？ RT ...,2019-01-01 09:55:56,0,"Qiita,Twitter",https://qiita.com/YumaInaura/items/2bad5f7dda7...,"エンジニア, ランキング, vqiita, contributions, 日本, エンジニア..."


In [18]:
len(df_text_ed)

8248

In [20]:
# 欠損値の数と割合を求める
def missingval_table(df):
    mv_df = pd.DataFrame(df.isnull().sum())
    mv_df = mv_df.rename(columns={0:'null count'})
    mv_df['null count(%)'] = mv_df['null count']/len(df)*100
    mv_df['variation'] = df.nunique().values
    mv_df['variation(%)'] = df.nunique().values/len(df)*100
    return mv_df

In [21]:
missingval_table(df_text_ed)

,null count,null count(%),variation,variation(%)
id,0,0.0,8248,100.000000
article_id,0,0.0,8248,100.000000
title,0,0.0,8241,99.915131
created_at,0,0.0,8232,99.806014
likes_count,0,0.0,210,2.546072
tags_str,0,0.0,6119,74.187682
url,0,0.0,8248,100.000000
sharing,0,0.0,8203,99.454413


In [22]:
df_text.to_csv("2019-01-01-content.csv")

In [12]:
# import numpy as np
# df1 = df_text.replace("", np.nan, regex=True)
# missingval_table(df1)

,null count,null count(%),variation,variation(%)
id,0,0.000000,8248,100.000000
text,18,0.218235,8205,99.478661
article_id,0,0.000000,8248,100.000000
created_at,0,0.000000,8232,99.806014
likes_count,0,0.000000,210,2.546072
similar_articles,8248,100.000000,0,0.000000
tags_str,0,0.000000,6119,74.187682
title,0,0.000000,8241,99.915131
url,0,0.000000,8248,100.000000


## dfに追加して、textを消去

In [30]:
df_text.to_csv("data/article/2019-01-01-content.csv")

In [13]:
from urllib.parse import urlparse
from sqlalchemy import create_engine
import pandas as pd

from sqlalchemy.sql import text
from datetime import datetime, date, timezone, timedelta

In [14]:

# url = urlparse(db_path) 
# conn = create_engine('mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8'.format(host = url.hostname, port=url.port, user = url.username, password= url.password, database = url.path[1:]),encoding='utf-8')
# table='raw_from_api'

# for i in range(len(date)-1):
#     print(" between'{}' and '{}';".format(date[i],date[i+1]))
#     sql_read=text("select * from qiita_db.raw_from_api where created_at between'{}' and '{}';".format(date[i],date[i+1]))
#     df = pd.read_sql(sql_read, conn)
    
#     share = []
#     for index, row in df.iterrows():
#         share.append(", ".join(get_shareing(row['html'])))
        
#     df_text = df
#     df_text["sharing"] = share
    
#     df_text_ed = df_text.drop(["html", "updated_at", "tokens", "user_id", "user_permanent_id", "comments_count", "similar_articles"], axis=1)
# #     print(df_text_ed.head(3))
#     df_text_ed.to_csv("data/article/{}-content.csv".format(date[i]))
    

In [15]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [18]:
from urllib.parse import urlparse
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.sql import text
from datetime import datetime, date, timezone, timedelta
import numpy as np


#dbserver_sampleの中身を書き換えて、ファイル名をdbserver.txtに変更
#dbserver.txtからdbのpathを読み込む

db_path = open('dbserver_sample.txt', 'r').read()

url = urlparse(db_path) 
conn = create_engine('mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8'.format(host = url.hostname, port=url.port, user = url.username, password= url.password, database = url.path[1:]),encoding='utf-8')
table='raw_from_api'

# start = '2018-12-01 00:00:00'
# end = '2020-01-01 00:00:00'

def make_csv_onemonth(start, end):
    date = pd.date_range(start, end,freq='M')
    
    for i in range(len(date)-1):
        logging.info(" between'{}' and '{}';".format(date[i],date[i+1]))
        sql_read=text("select * from qiita_db.raw_from_api where created_at between'{}' and '{}';".format(date[i],date[i+1]))
        df = pd.read_sql(sql_read, conn)
        df = df.replace("", np.nan, regex=True)
        
    
        share = []
        textss = []
        for index, row in df.iterrows():
            share.append(", ".join(get_shareing(row['html'])))
            textss.append(get_string(row['html']))
        
        df_text = df
        df_text["sharing"] = share
        df_text["text"] = textss
    
        df_text_ed = df_text.drop(["html", "updated_at", "tokens", "user_id", "user_permanent_id", "comments_count", "similar_articles"], axis=1)
        df_text_ed.to_csv("data/article/{}-content.csv".format(date[i]))
    
    

In [ ]:
start = '2017-01-01 00:00:00'
end = '2017-03-01 00:00:00'
make_csv_onemonth(start,end)

INFO:root: between'2017-01-31 00:00:00' and '2017-02-28 00:00:00';
